# Install and import required libraries

In [ ]:
!pip install flair

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import pandas as pd 
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import google.colab
from tqdm import tqdm
import re
import warnings
import csv

from flair.models import SequenceTagger
from collections import Counter
from flair.datasets import UD_INDONESIAN
from flair.embeddings import WordEmbeddings, StackedEmbeddings
from flair.models import SequenceTagger
from flair.trainers import ModelTrainer
from flair.data import Sentence

warnings.filterwarnings("ignore")

# Import data

In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [ ]:
df_tweets = pd.read_csv('/content/drive/MyDrive/TA/tweets/ruhutsitompul/done_labelled_ruhutsitompul.csv', delimiter=',', error_bad_lines=False)
df_tweets

,Unnamed: 0,ID,Tweet Time,Convo ID,Likes Count,Retweets Count,Tweets,Tweets(rendered),in_reply_to_tweet_id,in_reply_to,...,clean_tweets_len,Month,social,historical,dehumanization,accusation,attack,loyalty,threat,Tweets.1
0,0,1585739644011892736,2022-10-27 21:06:07,1585739644011892736,460,80,Sama banget Komentar ini dgn para pendukung se...,Sama banget Komentar ini dgn para pendukung se...,NaN,NaN,...,11,October,1,0,0,0,0.0,0,0,Sama banget Komentar ini dgn para pendukung se...
1,1,1585737538806853632,2022-10-27 20:57:45,1585737538806853632,420,92,Ha ha ha sigundul penguasa ancol karena selama...,Ha ha ha sigundul penguasa ancol karena selama...,NaN,NaN,...,15,October,1,0,0,1,0.0,0,0,Ha ha ha sigundul penguasa ancol karena selama...
2,2,1584780616369049605,2022-10-25 05:35:17,1584780616369049605,343,40,Kasihan Ibu ini jadi korban akibat dicuci otak...,Kasihan Ibu ini jadi korban akibat dicuci otak...,NaN,NaN,...,14,October,0,0,1,1,0.0,0,0,Kasihan Ibu ini jadi korban akibat dicuci otak...
3,3,1584778048120836096,2022-10-25 05:25:05,1584778048120836096,212,23,Ha ha hakadrun pada stresssssss mengenai beber...,Ha ha hakadrun pada stresssssss mengenai beber...,NaN,NaN,...,18,October,1,0,1,0,0.0,0,0,Ha ha hakadrun pada stresssssss mengenai beber...
4,4,1584774254502912000,2022-10-25 05:10:00,1584774254502912000,860,63,Kok sewot dgn Pidato Sambutan Bpk Joko Widodo ...,Kok sewot dgn Pidato Sambutan Bpk Joko Widodo ...,NaN,NaN,...,33,October,0,0,1,0,0.0,0,0,Kok sewot dgn Pidato Sambutan Bpk Joko Widodo ...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4178,4178,1118666888647602177,2019-04-18 00:05:44,1118666888647602177,0,0,@NediSetiadi Kami sedih krn biar sudah kalian ...,@NediSetiadi Kami sedih krn biar sudah kalian ...,NaN,NaN,...,14,April,1,0,0,1,0.0,0,0,@NediSetiadi Kami sedih krn biar sudah kalian ...
4179,4179,1118665503642247170,2019-04-18 00:00:14,1118665503642247170,7,0,@TanYoana haiya Amoy ini makin stresssssss kac...,@TanYoana haiya Amoy ini makin stresssssss kac...,NaN,NaN,...,5,April,0,0,0,0,0.0,0,0,@TanYoana haiya Amoy ini makin stresssssss kac...
4180,4180,1118664923301572608,2019-04-17 23:57:56,1118664923301572608,1684,200,Terima kasih KPU BAWASLU DKPP sbg Penyelenggar...,Terima kasih KPU BAWASLU DKPP sbg Penyelenggar...,NaN,NaN,...,31,April,0,0,0,0,0.0,0,0,Terima kasih KPU BAWASLU DKPP sbg Penyelenggar...
4181,4181,1118396091148460032,2019-04-17 06:09:41,1118396091148460032,6,0,"@panca66 Ha ha ha maaf memilih diri sendiri, s...","@panca66 Ha ha ha maaf memilih diri sendiri, s...",NaN,NaN,...,5,April,0,0,0,0,0.0,0,0,"@panca66 Ha ha ha maaf memilih diri sendiri, s..."


In [ ]:
# Remove tweets without any aspect
df_tweets = df_tweets.loc[(df_tweets[['social', 'historical', 'dehumanization', 'accusation', 'attack', 'loyalty', 'threat']] != 0).any(axis=1)]
df_tweets = df_tweets.reset_index(drop=True)
df_tweets

,Unnamed: 0,ID,Tweet Time,Convo ID,Likes Count,Retweets Count,Tweets,Tweets(rendered),in_reply_to_tweet_id,in_reply_to,...,clean_tweets_len,Month,social,historical,dehumanization,accusation,attack,loyalty,threat,Tweets.1
0,0,1585739644011892736,2022-10-27 21:06:07,1585739644011892736,460,80,Sama banget Komentar ini dgn para pendukung se...,Sama banget Komentar ini dgn para pendukung se...,NaN,NaN,...,11,October,1,0,0,0,0.0,0,0,Sama banget Komentar ini dgn para pendukung se...
1,1,1585737538806853632,2022-10-27 20:57:45,1585737538806853632,420,92,Ha ha ha sigundul penguasa ancol karena selama...,Ha ha ha sigundul penguasa ancol karena selama...,NaN,NaN,...,15,October,1,0,0,1,0.0,0,0,Ha ha ha sigundul penguasa ancol karena selama...
2,2,1584780616369049605,2022-10-25 05:35:17,1584780616369049605,343,40,Kasihan Ibu ini jadi korban akibat dicuci otak...,Kasihan Ibu ini jadi korban akibat dicuci otak...,NaN,NaN,...,14,October,0,0,1,1,0.0,0,0,Kasihan Ibu ini jadi korban akibat dicuci otak...
3,3,1584778048120836096,2022-10-25 05:25:05,1584778048120836096,212,23,Ha ha hakadrun pada stresssssss mengenai beber...,Ha ha hakadrun pada stresssssss mengenai beber...,NaN,NaN,...,18,October,1,0,1,0,0.0,0,0,Ha ha hakadrun pada stresssssss mengenai beber...
4,4,1584774254502912000,2022-10-25 05:10:00,1584774254502912000,860,63,Kok sewot dgn Pidato Sambutan Bpk Joko Widodo ...,Kok sewot dgn Pidato Sambutan Bpk Joko Widodo ...,NaN,NaN,...,33,October,0,0,1,0,0.0,0,0,Kok sewot dgn Pidato Sambutan Bpk Joko Widodo ...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1659,4151,1120875995962101760,2019-04-24 02:23:56,1120875995962101760,26,4,"@ustadtengkuzul Belajarlah menerima kekalahan,...","@ustadtengkuzul Belajarlah menerima kekalahan,...",NaN,NaN,...,12,April,1,0,0,1,0.0,0,0,"@ustadtengkuzul Belajarlah menerima kekalahan,..."
1660,4153,1120574450410217472,2019-04-23 06:25:42,1120574450410217472,1660,264,Aneh tapi Nyata ini bukan HOAX di Uk...,Aneh tapi Nyata ini bukan HOAX di Uk...,NaN,NaN,...,13,April,0,0,0,1,0.0,0,0,Aneh tapi Nyata ini bukan HOAX di Uk...
1661,4166,1119465028040527872,2019-04-20 04:57:15,1119465028040527872,1311,193,"Tim Sukses Pak Jokowi bukan Panglima Hukum, Pr...","Tim Sukses Pak Jokowi bukan Panglima Hukum, Pr...",NaN,NaN,...,24,April,1,0,0,1,0.0,0,0,"Tim Sukses Pak Jokowi bukan Panglima Hukum, Pr..."
1662,4168,1119429279811428353,2019-04-20 02:35:12,1119429279811428353,10,0,@FerdinandHutah2 Masih saja kau menggonggong s...,@FerdinandHutah2 Masih saja kau menggonggong s...,NaN,NaN,...,6,April,0,0,1,0,0.0,0,0,@FerdinandHutah2 Masih saja kau menggonggong s...


# POS tagging with flair

In [ ]:
# 1. get the corpus
corpus = UD_INDONESIAN().downsample(0.1)
print(corpus)

# 2. what label do we want to predict?
label_type = 'upos'

# 3. make the label dictionary from the corpus
label_dict = corpus.make_label_dictionary(label_type=label_type)
print(label_dict)

# 4. initialize embeddings
embedding_types = [

    WordEmbeddings('id-crawl'),
    WordEmbeddings('id'),

    # comment in this line to use character embeddings
    # CharacterEmbeddings(),

    # comment in these lines to use flair embeddings
    # FlairEmbeddings('news-forward'),
    # FlairEmbeddings('news-backward'),
]

embeddings = StackedEmbeddings(embeddings=embedding_types)

# 5. initialize sequence tagger
tagger = SequenceTagger(hidden_size=256,
                        embeddings=embeddings,
                        tag_dictionary=label_dict,
                        tag_type=label_type,
                        use_crf=True)

# 6. initialize trainer
trainer = ModelTrainer(tagger, corpus)

# 7. start training
trainer.train('resources/taggers/example-upos',
              learning_rate=0.1,
              mini_batch_size=32,
              max_epochs=10)

2023-03-13 07:41:36,577 Reading data from /root/.flair/datasets/ud_indonesian
2023-03-13 07:41:36,580 Train: /root/.flair/datasets/ud_indonesian/id_gsd-ud-train.conllu
2023-03-13 07:41:36,582 Dev: /root/.flair/datasets/ud_indonesian/id_gsd-ud-dev.conllu
2023-03-13 07:41:36,584 Test: /root/.flair/datasets/ud_indonesian/id_gsd-ud-test.conllu
2023-03-13 07:41:51,310 Warning: An empty Sentence was created! Are there empty strings in your dataset?
2023-03-13 07:41:52,384 Warning: An empty Sentence was created! Are there empty strings in your dataset?
2023-03-13 07:41:54,726 Warning: An empty Sentence was created! Are there empty strings in your dataset?
Corpus: 448 train + 56 dev + 56 test sentences
2023-03-13 07:41:54,742 Computing label dictionary. Progress:


448it [00:00, 6445.29it/s]

2023-03-13 07:41:54,865 Dictionary created for label 'upos' with 17 values: NOUN (seen 2145 times), PROPN (seen 1921 times), PUNCT (seen 1436 times), VERB (seen 1070 times), ADP (seen 932 times), PRON (seen 566 times), NUM (seen 428 times), ADJ (seen 354 times), ADV (seen 306 times), CCONJ (seen 289 times), DET (seen 270 times), AUX (seen 190 times), SCONJ (seen 187 times), PART (seen 94 times), SYM (seen 22 times), X (seen 11 times)
Dictionary with 17 tags: <unk>, NOUN, PROPN, PUNCT, VERB, ADP, PRON, NUM, ADJ, ADV, CCONJ, DET, AUX, SCONJ, PART, SYM, X


2023-03-13 07:42:24,414 SequenceTagger predicts: Dictionary with 17 tags: <unk>, NOUN, PROPN, PUNCT, VERB, ADP, PRON, NUM, ADJ, ADV, CCONJ, DET, AUX, SCONJ, PART, SYM, X
2023-03-13 07:42:24,486 ----------------------------------------------------------------------------------------------------
2023-03-13 07:42:24,492 Model: "SequenceTagger(
  (embeddings): StackedEmbeddings(
    (list_embedding_0): WordEmbeddings(
      'id-crawl'
      (embedding): Embedding(1000000, 300)
    )
    (list_embedding_1): WordEmbeddings(
      'id'
      (embedding): Embedding(300686, 300)
    )
  )
  (word_dropout): WordDropout(p=0.05)
  (locked_dropout): LockedDropout(p=0.5)
  (embedding2nn): Linear(in_features=600, out_features=600, bias=True)
  (rnn): LSTM(600, 256, batch_first=True, bidirectional=True)
  (linear): Linear(in_features=512, out_features=19, bias=True)
  (loss_function): ViterbiLoss()
  (crf): CRF()
)"
2023-03-13 07:42:24,493 --------------------------------------------------------------

100%|██████████| 2/2 [00:00<00:00,  6.48it/s]

...

2023-03-13 07:45:26,643 BAD EPOCHS (no improvement): 0
2023-03-13 07:45:26,646 saving best model
2023-03-13 07:45:43,323 ----------------------------------------------------------------------------------------------------
2023-03-13 07:45:51,262 SequenceTagger predicts: Dictionary with 19 tags: <unk>, NOUN, PROPN, PUNCT, VERB, ADP, PRON, NUM, ADJ, ADV, CCONJ, DET, AUX, SCONJ, PART, SYM, X, <START>, <STOP>


100%|██████████| 2/2 [00:00<00:00,  4.56it/s]

2023-03-13 07:45:52,791 Evaluating as a multi-label problem: False
2023-03-13 07:45:52,818 0.7822	0.7822	0.7822	0.7822


2023-03-13 07:45:52,819 
Results:
- F-score (micro) 0.7822
- F-score (macro) 0.6217
- Accuracy 0.7822

By class:
              precision    recall  f1-score   support

       PROPN     0.7054    0.9753    0.8187       243
        NOUN     0.7797    0.6834    0.7284       259
        VERB     0.8467    0.8467    0.8467       150
       PUNCT     0.9864    0.9932    0.9898       146
         ADP     0.8605    0.8409    0.8506       132
        PRON     0.8400    0.9844    0.9065        64
       CCONJ     0.8780    0.8571    0.8675        42
         ADV     0.3696    0.4857    0.4198        35
         ADJ     0.7059    0.2034    0.3158        59
         NUM     0.9200    0.7931    0.8519        29
         DET     0.4167    0.3448    0.3774        29
         AUX     0.7500    0.3214    0.4500        28
       SCONJ     0.3571    0.2273    0.2778        22
        PART     0.8333    0.5000    0.6250        10
           X     0.0000    0.0000    0.0000         1

    accuracy         

{'test_score': 0.7822257806244995,
 'dev_score_history': [0.3593073593073593,
  0.47878787878787876,
  0.5575757575757576,
  0.6216450216450217,
  0.6857142857142857,
  0.6961038961038961,
  0.7246753246753247,
  0.7541125541125541,
  0.7610389610389611,
  0.780952380952381],
 'train_loss_history': [2.6656418899164405,
  2.0629837189900084,
  1.6601365677893551,
  1.4410012598525708,
  1.2934544993151356,
  1.1636683064050117,
  1.0428368744114447,
  0.9513065930437334,
  0.8782457494068584,
  0.8175897384597275],
 'dev_loss_history': [2.298128843307495,
  1.7612498998641968,
  1.4412130117416382,
  1.2488306760787964,
  1.100416660308838,
  0.9665073156356812,
  0.8685756325721741,
  0.7814163565635681,
  0.716454029083252,
  0.6564066410064697]}

#3-gram

In [ ]:
# load the Flair POS tagger for Indonesian language
tagger = SequenceTagger.load('resources/taggers/example-upos/best-model.pt')

# Tokenize and filter tweets with category 1
tweets = df_tweets[df_tweets["social"] == 1]["clean_tweets"].tolist()

# Create trigrams
trigrams = []
for tweet in tweets:
    s = Sentence(tweet)
    tagger.predict(s)
    tags = [token.get_labels()[0].value for token in s.tokens]
    trigrams.extend(zip(tags, tags[1:], tags[2:]))

# Count trigrams and get top 3 most frequent ones
trigram_counts = Counter(trigrams)
top_n_trigrams = trigram_counts.most_common(10)

# Print top trigrams
for trigram, count in top_n_trigrams:
    print("Trigram: {} - Count: {}".format(trigram, count))

2023-03-13 07:45:58,196 SequenceTagger predicts: Dictionary with 19 tags: <unk>, NOUN, PROPN, PUNCT, VERB, ADP, PRON, NUM, ADJ, ADV, CCONJ, DET, AUX, SCONJ, PART, SYM, X, <START>, <STOP>
Trigram: ('PROPN', 'PROPN', 'PROPN') - Count: 3491
Trigram: ('PROPN', 'PROPN', 'VERB') - Count: 542
Trigram: ('NOUN', 'NOUN', 'NOUN') - Count: 525
Trigram: ('PROPN', 'VERB', 'NOUN') - Count: 399
Trigram: ('NOUN', 'PROPN', 'PROPN') - Count: 370
Trigram: ('VERB', 'NOUN', 'NOUN') - Count: 310
Trigram: ('NOUN', 'NOUN', 'ADJ') - Count: 235
Trigram: ('VERB', 'PROPN', 'PROPN') - Count: 222
Trigram: ('NOUN', 'NOUN', 'PROPN') - Count: 215
Trigram: ('NOUN', 'ADJ', 'PROPN') - Count: 180


In [ ]:
# load the Flair POS tagger for Indonesian language
tagger = SequenceTagger.load('resources/taggers/example-upos/best-model.pt')

# Tokenize and filter tweets with category 1
tweets = df_tweets[df_tweets["historical"] == 1]["clean_tweets"].tolist()

# Create trigrams
trigrams = []
for tweet in tweets:
    s = Sentence(tweet)
    tagger.predict(s)
    tags = [token.get_labels()[0].value for token in s.tokens]
    trigrams.extend(zip(tags, tags[1:], tags[2:]))

# Count trigrams and get top 3 most frequent ones
trigram_counts = Counter(trigrams)
top_n_trigrams = trigram_counts.most_common(10)

# Print top trigrams
for trigram, count in top_n_trigrams:
    print("Trigram: {} - Count: {}".format(trigram, count))

2023-03-13 07:46:13,826 SequenceTagger predicts: Dictionary with 19 tags: <unk>, NOUN, PROPN, PUNCT, VERB, ADP, PRON, NUM, ADJ, ADV, CCONJ, DET, AUX, SCONJ, PART, SYM, X, <START>, <STOP>
Trigram: ('PROPN', 'PROPN', 'PROPN') - Count: 213
Trigram: ('PROPN', 'PROPN', 'VERB') - Count: 26
Trigram: ('NOUN', 'NOUN', 'NOUN') - Count: 26
Trigram: ('PROPN', 'VERB', 'NOUN') - Count: 15
Trigram: ('NOUN', 'PROPN', 'PROPN') - Count: 15
Trigram: ('VERB', 'PROPN', 'PROPN') - Count: 13
Trigram: ('PROPN', 'PROPN', 'ADV') - Count: 12
Trigram: ('PROPN', 'VERB', 'PROPN') - Count: 12
Trigram: ('VERB', 'NOUN', 'NOUN') - Count: 10
Trigram: ('NOUN', 'NOUN', 'ADJ') - Count: 9


In [ ]:
# load the Flair POS tagger for Indonesian language
tagger = SequenceTagger.load('resources/taggers/example-upos/best-model.pt')

# Tokenize and filter tweets with category 1
tweets = df_tweets[df_tweets["dehumanization"] == 1]["clean_tweets"].tolist()

# Create trigrams
trigrams = []
for tweet in tweets:
    s = Sentence(tweet)
    tagger.predict(s)
    tags = [token.get_labels()[0].value for token in s.tokens]
    trigrams.extend(zip(tags, tags[1:], tags[2:]))

# Count trigrams and get top 3 most frequent ones
trigram_counts = Counter(trigrams)
top_n_trigrams = trigram_counts.most_common(10)

# Print top trigrams
for trigram, count in top_n_trigrams:
    print("Trigram: {} - Count: {}".format(trigram, count))

2023-03-13 07:46:20,184 SequenceTagger predicts: Dictionary with 19 tags: <unk>, NOUN, PROPN, PUNCT, VERB, ADP, PRON, NUM, ADJ, ADV, CCONJ, DET, AUX, SCONJ, PART, SYM, X, <START>, <STOP>
Trigram: ('PROPN', 'PROPN', 'PROPN') - Count: 3075
Trigram: ('NOUN', 'NOUN', 'NOUN') - Count: 391
Trigram: ('PROPN', 'PROPN', 'VERB') - Count: 365
Trigram: ('NOUN', 'PROPN', 'PROPN') - Count: 353
Trigram: ('PROPN', 'VERB', 'NOUN') - Count: 244
Trigram: ('NOUN', 'NOUN', 'ADJ') - Count: 235
Trigram: ('VERB', 'NOUN', 'NOUN') - Count: 209
Trigram: ('NOUN', 'NOUN', 'PROPN') - Count: 181
Trigram: ('VERB', 'PROPN', 'PROPN') - Count: 175
Trigram: ('PROPN', 'VERB', 'PROPN') - Count: 132


In [ ]:
# load the Flair POS tagger for Indonesian language
tagger = SequenceTagger.load('resources/taggers/example-upos/best-model.pt')

# Tokenize and filter tweets with category 1
tweets = df_tweets[df_tweets["accusation"] == 1]["clean_tweets"].tolist()

# Create trigrams
trigrams = []
for tweet in tweets:
    s = Sentence(tweet)
    tagger.predict(s)
    tags = [token.get_labels()[0].value for token in s.tokens]
    trigrams.extend(zip(tags, tags[1:], tags[2:]))

# Count trigrams and get top 3 most frequent ones
trigram_counts = Counter(trigrams)
top_n_trigrams = trigram_counts.most_common(10)

# Print top trigrams
for trigram, count in top_n_trigrams:
    print("Trigram: {} - Count: {}".format(trigram, count))

2023-03-13 07:46:34,599 SequenceTagger predicts: Dictionary with 19 tags: <unk>, NOUN, PROPN, PUNCT, VERB, ADP, PRON, NUM, ADJ, ADV, CCONJ, DET, AUX, SCONJ, PART, SYM, X, <START>, <STOP>
Trigram: ('PROPN', 'PROPN', 'PROPN') - Count: 2421
Trigram: ('PROPN', 'PROPN', 'VERB') - Count: 357
Trigram: ('NOUN', 'NOUN', 'NOUN') - Count: 324
Trigram: ('PROPN', 'VERB', 'NOUN') - Count: 245
Trigram: ('NOUN', 'PROPN', 'PROPN') - Count: 238
Trigram: ('VERB', 'NOUN', 'NOUN') - Count: 177
Trigram: ('VERB', 'PROPN', 'PROPN') - Count: 165
Trigram: ('NOUN', 'NOUN', 'ADJ') - Count: 153
Trigram: ('NOUN', 'NOUN', 'PROPN') - Count: 135
Trigram: ('PROPN', 'VERB', 'PROPN') - Count: 117
